In [1]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Flatten, Dense
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import backend as K
import keras

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

Using TensorFlow backend.


In [2]:
img_width, img_height = 31, 31

nb_epoch = 50
batch_size = 32

In [3]:
base_dir = '/scratch/image_datasets/1_for_learned_sift/ready'

train_data_dir      = base_dir + '/patches/train'
validation_data_dir = base_dir + '/patches/validation'
test_data_dir       = base_dir + '/patches/test'

train_descrs_dir      = base_dir + '/descriptors_angles/train'
validation_descrs_dir = base_dir + '/descriptors_angles/validation'
test_descrs_dir       = base_dir + '/descriptors_angles/test'

In [4]:
def loading_data(dir_patches, dir_descrs):
    files_patches = listdir(dir_patches + '/class0')
    files_patches.sort()
    
    files_descrs = listdir(dir_descrs + '/class0')
    files_descrs.sort()
    
    assert len(files_patches) == len(files_descrs), "The number of patches doesn't match the number of descriptors."

    patches = []
    descrs = []

    
    for file_patch, file_descr in zip(files_patches, files_descrs):
        patch = imageio.imread(dir_patches + '/class0/' + file_patch)
#         print(patch.shape)
        if patch.shape[0] == 31:
            patches.append(patch)
            descr = np.load(dir_descrs + '/class0/' + file_descr)
            descrs.append(descr)
#         elif image.shape[0] == 19:
#             temp_count19 += 1
        
        

    patches = np.array(patches)
    patches = patches.astype(np.float64) / 255
    
    descrs = np.array(descrs)
    descrs = descrs.astype(np.float64) / 255
    print("patches", patches.shape, "  descrs", descrs.shape)
    
    return patches, descrs

In [5]:
x_train, y_train = loading_data(train_data_dir, train_descrs_dir)
x_validation, y_validation = loading_data(validation_data_dir, validation_descrs_dir)

patches (3382, 31, 31)   descrs (3382, 129)
patches (409, 31, 31)   descrs (409, 129)


In [6]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[2], x_train.shape[2], 1))
x_validation = x_validation.reshape((x_validation.shape[0], x_validation.shape[2], x_validation.shape[2], 1))

In [7]:
x_validation.shape

(409, 31, 31, 1)

In [26]:
def MSLE_plus_plus(y_true, y_pred):
    if not K.is_tensor(y_pred):
        y_pred = K.constant(y_pred)
    y_true = K.cast(y_true, y_pred.dtype)
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    custom_loss_log = K.log(K.clip(y_true + y_pred, K.epsilon(), None) + 1.)
    custom_loss_denominator = (y_true * y_pred + 0.005) * 256  # parameters to be further adjusted
    return K.mean(K.square(first_log - second_log) + custom_loss_log / custom_loss_denominator, axis=-1)

In [8]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="relu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="relu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="sigmoid")(x)

encoder = Model(input_img, encoded)

encoder.compile(optimizer='adadelta', loss="binary_crossentropy")
#next up: encoder.compile(optimizer='sgd', metrics=['categorical_accuracy'], loss='categorical_crossentropy')


encoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 16)          0   

In [9]:
def fixed_generator(x_train, y_train, batch_size):
    while True:
        batch_list_x = []
        batch_list_y = []
        
        for i in range(x_train.shape[0]):
            batch_list_x.append(x_train[i])
            batch_list_y.append(y_train[i])
            if len(batch_list_x) == batch_size:
                yield (np.array(batch_list_x),np.array(batch_list_y))
                batch_list_x = []
                batch_list_y = []


In [ ]:
model_version = '2.0.0.0_encoder_lastsigmoid_bce_adadelta'
encoder = load_model(base_dir + '/ae' + model_version + '.h5')

In [21]:
model_version = '2.0.0.2_encoder_lastsigmoid_bce_adadelta_moreepochs400'

os.system('mkdir ' + base_dir + '/weights' + model_version)
checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder.fit_generator(fixed_generator(x_train, y_train, 32),
                steps_per_epoch=3382,
                epochs=400,
                validation_data=fixed_generator(x_validation, y_validation, 32),
                validation_steps=409,
                callbacks=[checkpointer]
                )
encoder.save(base_dir + '/ae' + model_version + '.h5')

# import keras.losses
# keras.losses.MSLE_plus_plus = MSLE_plus_plus
# encoder = load_model(base_dir + '/ae' + model_version + '.h5')

Epoch 1/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2930 - val_loss: 0.3791
Epoch 2/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2930 - val_loss: 0.3732
Epoch 3/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2929 - val_loss: 0.4069
Epoch 4/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2930 - val_loss: 0.3920
Epoch 5/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2929 - val_loss: 0.3940
Epoch 6/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2929 - val_loss: 0.3600
Epoch 7/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2928 - val_loss: 0.3750
Epoch 8/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2927 - val_loss: 0.4113
Epoch 9/400
3382/3382 [==============================] - 68s 20ms/step - loss: 0.2929 - val_loss: 0.3574
Epoch 10/400
3382/3382 [==============================]

3382/3382 [==============================] - 67s 20ms/step - loss: 0.2914 - val_loss: 0.3674
Epoch 79/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2913 - val_loss: 0.3812
Epoch 80/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2914 - val_loss: 0.4215
Epoch 81/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2915 - val_loss: 0.3615
Epoch 82/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2914 - val_loss: 0.4210
Epoch 83/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2914 - val_loss: 0.4412
Epoch 84/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2913 - val_loss: 0.3944
Epoch 85/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2914 - val_loss: 0.3940
Epoch 86/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2914 - val_loss: 0.3834
Epoch 87/400
3382/3382 [==============================] - 6

Epoch 155/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2907 - val_loss: 0.4581
Epoch 156/400
3382/3382 [==============================] - 68s 20ms/step - loss: 0.2906 - val_loss: 0.4080
Epoch 157/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2907 - val_loss: 0.4029
Epoch 158/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2906 - val_loss: 0.3900
Epoch 159/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2906 - val_loss: 0.4286
Epoch 160/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2906 - val_loss: 0.4117
Epoch 161/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2906 - val_loss: 0.4183
Epoch 162/400
3382/3382 [==============================] - 68s 20ms/step - loss: 0.2907 - val_loss: 0.3722
Epoch 163/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2906 - val_loss: 0.3950
Epoch 164/400
3382/3382 [============

3382/3382 [==============================] - 67s 20ms/step - loss: 0.2903 - val_loss: 0.4389
Epoch 232/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2902 - val_loss: 0.4128
Epoch 233/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2903 - val_loss: 0.4183
Epoch 234/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2902 - val_loss: 0.3779
Epoch 235/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2902 - val_loss: 0.3933
Epoch 236/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2902 - val_loss: 0.4374
Epoch 237/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2901 - val_loss: 0.3620
Epoch 238/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2903 - val_loss: 0.4326
Epoch 239/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2902 - val_loss: 0.4535
Epoch 240/400
3382/3382 [==========================

3382/3382 [==============================] - 67s 20ms/step - loss: 0.2900 - val_loss: 0.3973
Epoch 308/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2899 - val_loss: 0.4394
Epoch 309/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2900 - val_loss: 0.3631
Epoch 310/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2900 - val_loss: 0.4440
Epoch 311/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2899 - val_loss: 0.4626
Epoch 312/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2900 - val_loss: 0.4139
Epoch 313/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2899 - val_loss: 0.4046
Epoch 314/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2900 - val_loss: 0.3929
Epoch 315/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2900 - val_loss: 0.4333
Epoch 316/400
3382/3382 [==========================

3382/3382 [==============================] - 67s 20ms/step - loss: 0.2897 - val_loss: 0.4543
Epoch 384/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2897 - val_loss: 0.4135
Epoch 385/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2897 - val_loss: 0.4084
Epoch 386/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2897 - val_loss: 0.3948
Epoch 387/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2898 - val_loss: 0.4375
Epoch 388/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2897 - val_loss: 0.4164
Epoch 389/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2897 - val_loss: 0.4294
Epoch 390/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2897 - val_loss: 0.3855
Epoch 391/400
3382/3382 [==============================] - 67s 20ms/step - loss: 0.2898 - val_loss: 0.4026
Epoch 392/400
3382/3382 [==========================

In [22]:
np.around(encoder.predict(imageio.imread(test_data_dir + "/class0/patch000013.bmp").reshape(1,31,31,1))*255)

array([[255., 255., 255.,   0.,   0.,   0., 255., 255., 255., 255., 195.,
          0.,   0.,   0., 255.,   0., 255., 255.,   0., 255.,   0.,   0.,
          0.,   0.,   0.,   0.,   0., 255., 255., 255.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0., 255., 255., 255., 255.,   0.,   0.,
          0.,   0., 255.,   0., 255.,   6.,   0.,   0., 255.,   0.,   0.,
          0.,   0.,   0.,   0., 255., 255.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0., 255., 255.,   0.,   0.,   0.,   0.,   0.,
          0., 255., 255., 255.,   0.,   0.,   0.,   0., 255., 255., 255.,
        255.,   0.,   0.,   0., 255., 255., 255., 255.,   0.,   0., 255.,
        255.,   0.,   0.,   0.,   0.,   0., 255., 255., 255.,   0.,   0.,
          0.,   0., 255., 255., 255.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0., 255.,   0.,   0., 255.,   0., 255.]], dtype=float32)

In [23]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1))*255)

array([[255., 255., 255., 255.,   0.,   0.,   0.,   0.,   0., 255., 255.,
        255., 255.,   0.,   0.,   0., 255., 255.,   0., 255., 255., 255.,
          0.,   0.,   0.,   0.,   0.,   0., 255., 255.,   0.,   0.,   0.,
         58., 255., 255., 255.,   0.,   0.,   0.,   0., 255., 255., 255.,
        255.,   0.,   0.,   0.,   0., 255.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0., 255., 255.,   0.,   0., 255., 255.,
        255., 255., 255.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 255.,
        255., 255., 255.,   0.,   0.,   0.,   0.,   0., 255., 255., 255.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 255., 255., 255.,
          0.,   0.,   0.,   0., 255.,   0., 255., 255.,   0.,   0., 255.,
        255.,   0.,   0.,   0.,   0.,   0.,   0., 255., 255.,   0.,   0.,
          0.,   0.,   0.,   0., 194., 255.,   0.,   0.]], dtype=float32)

In [45]:
temp_y_prime = encoder.predict(imageio.imread("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_train16/class0/patch000012.png").reshape(1,16,16,3))

In [46]:
temp_y = np.load("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_sifts/tiny_train16/class0/patch000012.npy")

In [47]:
print(np.corrcoef(temp_y, temp_y_prime[0])[1,0], "\n")
for i in range(temp_y_prime.shape[1]):
    print('{:>12f}{:>12f}'.format(temp_y[i], temp_y_prime[0, i]))      
#     print(temp_y_prime[0, i], temp_y[i])

0.6393183138370047 

    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    

In [21]:
np.round(temp, decimals=9)

array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 3.427e-06, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+0

In [14]:
import tensorflow as tf

print(tf.__version__)

1.14.0


In [19]:
tf.disable_v2_behavior()

W0122 20:00:43.071549 140376228677440 deprecation.py:323] From /scratch/tensorflow/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [21]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)